In [ ]:
!pip install -q langchain langchain_google_genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 651.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 6.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [ ]:
import google.colab.userdata as userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.tools import tool
print("Libraries installed and imported successfully.")

Libraries installed and imported successfully.


In [ ]:
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    print("Google API Key loaded successfully.")
except Exception as e:
    print("Could not load Google API Key. Please follow the instructions to add it to Colab's secrets.")
    # Stop execution if the key is not found.
    raise e


Google API Key loaded successfully.


In [ ]:
@tool
def say_hi(query: str) -> str:
    """
    Use this tool when the user greets you, says hello, or starts a conversation.
    Example prompts: 'Hi', 'Hello there', 'How are you?'
    """
    return "Hi"

@tool
def say_bye(query: str) -> str:
    """
    Use this tool when the user wants to end the conversation or says goodbye.
    Example prompts: 'Goodbye', 'See you later', 'Bye now'
    """
    return "Bye"

tools = [say_hi, say_bye]
print("Tools defined: say_hi, say_bye")

Tools defined: say_hi, say_bye


In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GOOGLE_API_KEY
    )

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an assistant. Your only job is to decide whether to use the 'say_hi' tool or the 'say_bye' tool based on the user's input."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
print("Agent created and ready to go!")

Agent created and ready to go!


In [ ]:

print("\n--- Running Test Case 1: Greeting ---")
response_hi = agent_executor.invoke({"input": "Hello"})
#print (response_hi['output'])
print("\nFinal Answer:", response_hi['output'])


--- Running Test Case 1: Greeting ---


> Entering new AgentExecutor chain...

Invoking: `say_hi` with `{'query': 'Hello'}`


HiOK. I used the `say_hi` tool.


> Finished chain.

Final Answer: OK. I used the `say_hi` tool.



In [ ]:
print("\n\n--- Running Test Case 2: Farewell ---")
response_bye = agent_executor.invoke({"input": "Ok, goodbye for now."})
print("\nFinal Answer:", response_bye['output'])



--- Running Test Case 2: Farewell ---


> Entering new AgentExecutor chain...

Invoking: `say_bye` with `{'query': 'Ok, goodbye for now.'}`


ByeGoodbye!


> Finished chain.

Final Answer: Goodbye!



In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool, Tool, AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

@tool
def greet_user(input: str) -> str:
    """Greets the user when they say hello or hi."""
    return "Hello!"

@tool
def farewell_user(input: str) -> str:
    """Says goodbye when the user ends the conversation."""
    return "Goodbye!"

tools = [greet_user, farewell_user]

prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a friendly assistant that greets or says goodbye using tools."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY)
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

print(agent_executor.invoke({"input": "Hi assistant!"}))
print(agent_executor.invoke({"input": "See you soon!"}))




> Entering new AgentExecutor chain...

Invoking: `greet_user` with `{'input': 'Hi assistant!'}`


Hello!Hello!


> Finished chain.
{'input': 'Hi assistant!', 'output': 'Hello!\n'}


> Entering new AgentExecutor chain...

Invoking: `farewell_user` with `{'input': 'See you soon!'}`


Goodbye!Goodbye!


> Finished chain.
{'input': 'See you soon!', 'output': 'Goodbye!\n'}
